# CV-703, Assignment 1

## Imports

In [ ]:
import numpy as np
import math
from tqdm import tqdm
import PIL
from PIL import Image
import torch
import torch.optim as optim
import torchvision
from torchvision import transforms
import torchvision.transforms as T
import pandas as pd
from models_to_finetune import deit_small_patch16_224, deit_base_patch16_224
from datasets import CUBDataset, DOGDataset, FOODDataset
import sys
import os

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

# Augmentations

In [ ]:
#CenterCrop
data_transform = transforms.Compose([
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])

#RandomHorizontal flip
data_transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])

#RandomRotation
data_transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomRotation(30),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])

# Loss functions

In [ ]:
#Label smoothing definition
class LabelSmoothingLoss(torch.nn.Module): 
    def __init__(self, classes=5, smoothing=0.0, dim=-1): 
        super(LabelSmoothingLoss, self).__init__() 
        self.confidence = 1.0 - smoothing 
        self.smoothing = smoothing 
        self.cls = classes 
        self.dim = dim 
    def forward(self, pred, target): 
        pred = pred.log_softmax(dim=self.dim) 
        with torch.no_grad():
            true_dist = torch.zeros_like(pred) 
            true_dist.fill_(self.smoothing / (self.cls - 1)) 
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence) 
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))
#To use: criterion = LabelSmoothingLoss()


#Focal loss definition
class FocalLoss(torch.nn.Module):
    def __init__(self, alpha=1, gamma=2, reduce=True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduce = reduce

    def forward(self, inputs, targets):
        BCE_loss = torch.nn.CrossEntropyLoss()(inputs, targets)

        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduce:
            return torch.mean(F_loss)
        else:
            return F_loss
#To use: criterion = FocalLoss()

# ResNet50 + DeiT model

In [ ]:
class MyEnsembleModel(torch.nn.Module):

    def __init__(self):

        super(MyModel,self).__init__()

        resnet = models.resnet50(pretrained=True)
        resnet_features = resnet.fc.in_features

        self.backbone1 = torch.nn.Sequential(*(list(resnet.children())[:-1]))

        transformer = deit_small_patch16_224(pretrained=True, use_top_n_heads=6,use_patch_outputs=False)
        transformer_features = transformer.head.in_features

        self.backbone2 = transformer

        for name,param in resnet.named_parameters():
            param.requires_grad = False

        for name,param in transformer.named_parameters():
            param.requires_grad = False

        self.resnet_mlp = torch.nn.Linear(in_features=resnet_features, out_features=1024)
        self.transformer_mlp = torch.nn.Linear(in_features=1000, out_features=1024)
        
        #change no.of classes here
        self.common = torch.nn.Linear(in_features=1024, out_features=320)


    def forward(self,x):

        resnet_out1 = self.backbone1(x)

        transformer_out1 = self.backbone2(x,fine_tune=True)

        resnet_out2 = self.resnet_mlp(resnet_out1.reshape(resnet_out1.shape[0],-1))

        transformer_out2 = self.transformer_mlp(transformer_out1)

        outt = self.common(0.5*resnet_out2+0.5*transformer_out2)

        return out
    
#To use: model = MyEnsembleModel()

## Setup dataset

### CUB-200-2011 (Birds)

In [ ]:
classes_number = 200

In [ ]:
class CUBDataset(torchvision.datasets.ImageFolder):
    """
    Dataset class for CUB Dataset
    """

    def __init__(self, image_root_path, caption_root_path=None, split="train", *args, **kwargs):
        """
        Args:
            image_root_path:      path to dir containing images and lists folders
            caption_root_path:    path to dir containing captions
            split:          train / test
            *args:
            **kwargs:
        """
        image_info = self.get_file_content(f"{image_root_path}/images.txt")
        self.image_id_to_name = {y[0]: y[1] for y in [x.strip().split(" ") for x in image_info]}
        split_info = self.get_file_content(f"{image_root_path}/train_test_split.txt")
        self.split_info = {self.image_id_to_name[y[0]]: y[1] for y in [x.strip().split(" ") for x in split_info]}
        self.split = "1" if split == "train" else "0"
        self.caption_root_path = caption_root_path

        super(CUBDataset, self).__init__(root=f"{image_root_path}/images", is_valid_file=self.is_valid_file,
                                         *args, **kwargs)

    def is_valid_file(self, x):
        return self.split_info[(x[len(self.root) + 1:])] == self.split

    @staticmethod
    def get_file_content(file_path):
        with open(file_path) as fo:
            content = fo.readlines()
        return content

In [ ]:
data_root = "/apps/local/shared/CV703/datasets/CUB/CUB_200_2011/"

mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)


# Write data transform here as per the requirement
data_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])

train_dataset = CUBDataset(image_root_path=f"{data_root}", transform=data_transform, split="train")
test_dataset = CUBDataset(image_root_path=f"{data_root}", transform=data_transform, split="test")
print('Number of train samples:', len(train_dataset))
print('Number of test samples:', len(test_dataset))


# Load in into the torch dataloader to get variable batch size, shuffle 
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, drop_last=True, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, drop_last=False, shuffle=True)

In [ ]:
len(train_loader), len(test_loader)

In [ ]:
for i, (inputs, labels) in enumerate(train_loader):
    print(inputs.shape)
    print(labels)
    print('='*50)
    break

### Stanford Dogs

In [ ]:
classes_number = 120

In [ ]:
class DOGDataset(ImageFolder):
    """
    Dataset class for CUB Dataset
    """

 

    def __init__(self, image_root_path, caption_root_path=None, split="train", *args, **kwargs):
        """
        Args:
            image_root_path:      path to dir containing images and lists folders
            caption_root_path:    path to dir containing captions
            split:          train / test
            *args:
            **kwargs:
        """
        image_info = self.get_file_content(f"{image_root_path}splits/file_list.mat")
        image_files = [o[0][0] for o in image_info]
        
        split_info = self.get_file_content(f"{image_root_path}/splits/{split}_list.mat")
        split_files = [o[0][0] for o in split_info]
        self.split_info = {}
        if split == 'train' :
            for image in image_files:
                if image in split_files:
                    self.split_info[image] = "1"
                else:
                    self.split_info[image] = "0"
        elif split== 'test' :
            for image in image_files:
                if image in split_files:
                    self.split_info[image] = "0"
                else:
                    self.split_info[image] = "1"
                    
        self.split = "1" if split == "train" else "0"
        self.caption_root_path = caption_root_path

 

        super(DOGDataset, self).__init__(root=f"{image_root_path}Images", is_valid_file = self.is_valid_file,
                                         *args, **kwargs)

 

    def is_valid_file(self, x):
        return self.split_info[(x[len(self.root) + 1:])] == self.split

 

    @staticmethod
    def get_file_content(file_path):
        content =  scipy.io.loadmat(file_path)
        return content['file_list']

In [ ]:
data_root = "/apps/local/shared/CV703/datasets/dog/"


mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

data_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])


train_dataset = DOGDataset(image_root_path=f"{data_root}", transform=data_transform, split="train")
test_dataset = DOGDataset(image_root_path=f"{data_root}", transform=data_transform, split="test")
print('Number of train samples:', len(train_dataset))
print('Number of test samples:', len(test_dataset))

# Load in into the torch dataloader to get variable batch size, shuffle 
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, drop_last=True, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, drop_last=False, shuffle=True)

In [ ]:
len(train_loader), len(test_loader)

In [ ]:
for i, (inputs, labels) in enumerate(test_loader):
    print(inputs.shape)
    print(labels)
    print('='*50)
    break

### CUB-200-2011 + Stanford Dog

In [ ]:
classes_number = 320

In [ ]:
class CUBDataset(torchvision.datasets.ImageFolder):
    """
    Dataset class for CUB Dataset
    """

    def __init__(self, image_root_path, caption_root_path=None, split="train", *args, **kwargs):
        """
        Args:
            image_root_path:      path to dir containing images and lists folders
            caption_root_path:    path to dir containing captions
            split:          train / test
            *args:
            **kwargs:
        """
        image_info = self.get_file_content(f"{image_root_path}/images.txt")
        self.image_id_to_name = {y[0]: y[1] for y in [x.strip().split(" ") for x in image_info]}
        split_info = self.get_file_content(f"{image_root_path}/train_test_split.txt")
        self.split_info = {self.image_id_to_name[y[0]]: y[1] for y in [x.strip().split(" ") for x in split_info]}
        self.split = "1" if split == "train" else "0"
        self.caption_root_path = caption_root_path

        super(CUBDataset, self).__init__(root=f"{image_root_path}/images", is_valid_file=self.is_valid_file,
                                         *args, **kwargs)

    def is_valid_file(self, x):
        return self.split_info[(x[len(self.root) + 1:])] == self.split

    @staticmethod
    def get_file_content(file_path):
        with open(file_path) as fo:
            content = fo.readlines()
        return content

In [ ]:
class DOGDataset(torchvision.datasets.ImageFolder):
    """
    Dataset class for DOG Dataset
    """

    def __init__(self, image_root_path, caption_root_path=None, split="train", *args, **kwargs):
        """
        Args:
            image_root_path:      path to dir containing images and lists folders
            caption_root_path:    path to dir containing captions
            split:          train / test
            *args:
            **kwargs:
        """
        image_info = self.get_file_content(f"{image_root_path}splits/file_list.mat")
        image_files = [o[0][0] for o in image_info]
        
        split_info = self.get_file_content(f"{image_root_path}/splits/{split}_list.mat")
        split_files = [o[0][0] for o in split_info]
        self.split_info = {}
        if split == 'train' :
            for image in image_files:
                if image in split_files:
                    self.split_info[image] = "1"
                else:
                    self.split_info[image] = "0"
        elif split== 'test' :
            for image in image_files:
                if image in split_files:
                    self.split_info[image] = "0"
                else:
                    self.split_info[image] = "1"
                    
        self.split = "1" if split == "train" else "0"
        self.caption_root_path = caption_root_path

        super(DOGDataset, self).__init__(root=f"{image_root_path}Images", is_valid_file = self.is_valid_file,
                                         *args, **kwargs)
        
        ## modify class index as we are going to concat to first dataset
        self.class_to_idx = {class_: idx+200 for idx, class_ in enumerate(self.class_to_idx)}
        
    def is_valid_file(self, x):
        return self.split_info[(x[len(self.root) + 1:])] == self.split
    
    def __getitem__(self, index):
        path, target = self.imgs[index]
        img = Image.open(os.path.join(path)).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        if self.target_transform is not None:
            target = self.target_transform(target)
        
        ## modify target class index as we are going to concat to first dataset
        return img, target + 200

    @staticmethod
    def get_file_content(file_path):
        content =  scipy.io.loadmat(file_path)
        return content['file_list']

In [ ]:
# CUB:
data_root_bird = "/apps/local/shared/CV703/datasets/CUB/CUB_200_2011/"

mean_bird = (0.485, 0.456, 0.406)
std_bird = (0.229, 0.224, 0.225)


# write data transform here as per the requirement
data_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean_bird, std=std_bird)
    ])

train_dataset_cub = CUBDataset(image_root_path=f"{data_root_bird}", transform=data_transform, split="train")
test_dataset_cub = CUBDataset(image_root_path=f"{data_root_bird}", transform=data_transform, split="test")
print('Number of train samples:', len(train_dataset_cub))
print('Number of test samples:', len(test_dataset_cub))

In [ ]:
# Dog:
mean_dog = (0.485, 0.456, 0.406)
std_dog = (0.229, 0.224, 0.225)

data_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean_dog, std=std_dog)
    ])


data_root_dog = "/apps/local/shared/CV703/datasets/dog/"

train_dataset_dog = DOGDataset(image_root_path=f"{data_root_dog}", transform=data_transform, split="train")
test_dataset_dog = DOGDataset(image_root_path=f"{data_root_dog}", transform=data_transform, split="test")
print('Number of train samples:', len(train_dataset_dog))
print('Number of test samples:', len(test_dataset_dog))

In [ ]:
# concatenated dataloader for CUB and DOG

train_loader = torch.utils.data.DataLoader(
             torch.utils.data.ConcatDataset([train_dataset_cub, train_dataset_dog]),
             batch_size=32, shuffle=True,
             num_workers=1, pin_memory=True)

test_loader = torch.utils.data.DataLoader(
             torch.utils.data.ConcatDataset([test_dataset_cub, test_dataset_dog]),
             batch_size=32, shuffle=True,
             num_workers=1, pin_memory=True)

In [ ]:
len(train_dataset_cub), len(train_dataset_dog), len(train_loader)

In [ ]:
len(test_dataset_cub), len(test_dataset_dog), len(test_loader)

In [ ]:
for i, (inputs, targets) in enumerate(train_loader):

    print('image :: ', inputs.shape)
    print(targets)
    break

### FoodX-251 Dataset

In [3]:
classes_number = 251

In [4]:
ds_type = "local" # comment out if using ds from the shared folder
#ds_type = "shared" # comment out if using ds from the local folder


if (ds_type == "local"):
    data_dir = "/home/dmitry.demidov/Documents/Datasets/FoodX-251"

    split = 'train'
    train_df = pd.read_csv(f'{data_dir}/annot/{split}_info.csv', names= ['image_name','label'])
    train_df['path'] = train_df['image_name'].map(lambda x: os.path.join(f'{data_dir}/{split}/{split}_set/', x))

    split = 'val'
    test_df = pd.read_csv(f'{data_dir}/annot/{split}_info.csv', names= ['image_name','label'])
    test_df['path'] = test_df['image_name'].map(lambda x: os.path.join(f'{data_dir}/{split}/{split}_set/', x))

elif (ds_type == "shared"):
    data_dir = "/apps/local/shared/CV703/datasets/FoodX/food_dataset"

    split = 'train'
    train_df = pd.read_csv(f'{data_dir}/annot/{split}_info.csv', names= ['image_name','label'])
    train_df['path'] = train_df['image_name'].map(lambda x: os.path.join(f'{data_dir}/{split}_set/', x))

    split = 'val'
    test_df = pd.read_csv(f'{data_dir}/annot/{split}_info.csv', names= ['image_name','label'])
    test_df['path'] = test_df['image_name'].map(lambda x: os.path.join(f'{data_dir}/{split}_set/', x))

else:
    print("ERROR: Choose dataset type (local/shared)!")

In [5]:
train_dataset = FOODDataset(train_df)
test_dataset = FOODDataset(test_df)
print('Number of train samples:', len(train_dataset))
print('Number of test samples:', len(test_dataset))

# load in into the torch dataloader to get variable batch size, shuffle 
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=256, drop_last=True, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, drop_last=False, shuffle=True) # Not enough memory for more than 32

Number of train samples: 118475
Number of test samples: 11994


In [6]:
# Print some statistics about the dataset:

print(len(train_dataset), len(test_dataset))

print(len(train_loader), len(test_loader))

for i, (inputs, labels) in enumerate(train_loader):
    print(inputs.shape)
    print(labels)
    print('='*50)

    break

118475 11994
462 375
torch.Size([256, 3, 224, 224])
tensor([100,  60,  87, 152,  25,  95, 179, 210, 187, 165,  65,  69, 178,   4,
        218, 207, 128,  55, 250,  94, 246,  44,  64, 146,  85,  13,  17, 215,
         81,   9,  39,  91,  28,  17, 148, 113, 232, 151, 128, 116,  58, 166,
        176,  94, 247,  79, 157, 129,  25, 147,  39, 237,   7, 113,  21, 155,
        230, 168,  70, 190, 214,  21, 160,  35, 150, 106, 124, 194, 152, 228,
        136, 198,  94,  30, 214, 249, 190,  31,  98, 221,  19, 115, 139,  74,
        142,  13, 138, 177, 178,  37, 156,  74, 122, 105,   6, 132, 205,  11,
        212, 245,  89, 217,  18, 212, 226,   3,  74,  68, 123, 171, 181, 152,
        249,  58, 143, 247, 202, 113,  39, 243,  55,  40, 147,  88,  12,  38,
        250,  30, 220,  89,  50, 146, 107, 139, 163, 134,  42,  37,  93, 123,
         13,  39, 148,  95, 197,  70,   0,  56,   8,  84, 201, 131,  98, 161,
          0, 196, 132, 244, 227, 143, 172,  35, 176, 133, 142, 241, 163,  94,
         99,

# Training + Evaluation

### Train, test funcs:

In [7]:
def train(device, model, criterion, optimizer, train_dataset, train_loader, epoch):
    print('Training....')
    
    model.train()

    epoch_loss = 0.0
    acc=0.0

    with tqdm(train_loader) as p_bar:
        for samples, targets in p_bar:
            samples = samples.to(device)
            targets = targets.to(device)
            
            #For ResNet50+DeiT ensemble model, change: model(samples)
            outputs = model(samples, fine_tune=True) 
            loss = criterion(outputs, targets)
            loss_value = loss.item()
            if not math.isfinite(loss_value):
                print("Loss is {}, stopping training".format(loss_value))
                sys.exit(1)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += outputs.shape[0] * loss.item()

            acc+=torch.sum(outputs.argmax(dim=-1) == targets).item()

    loss_print = epoch_loss / len(train_dataset)
    acc_print = acc/len(train_dataset)
    epoch_print = epoch+1

    print("Epoch:", epoch_print, "|", "Loss:", loss_print)
    print("Train Accuracy:{0:.3%}".format(acc_print))

    f = open("./models/statistics.txt", "a")
    text_train = "Epoch: " + str(epoch_print) + ", " + "Train Loss: " + str(loss_print) + ", " + "Train Accuracy: " + str(acc_print) + "\n"
    f.write(text_train)
    f.close()
    
    torch.save({'state_dict': model.state_dict()}, './models/model_{0}ep_{1:.2}loss.pt'.format(epoch_print, loss_print))

    sent_results(text=text_train)

    #del samples
    #del targets



def test(device, model, criterion, test_dataset, test_loader, model_path = './folder/path.pt', test_only = False):
    print('Testing....')

    if test_only:
        print('Test only!')
        state_dict = torch.load(model_path)['state_dict']
        model.load_state_dict(state_dict)
        model = model.to(device)

    model.eval()

    epoch_loss = 0.0
    acc=0.0

    with tqdm(test_loader) as p_bar:
        for samples, targets in p_bar:
            samples = samples.to(device)
            targets = targets.to(device)
            
            #For ResNet50+DeiT ensemble model, change: model(samples)
            outputs = model(samples, fine_tune=False)

            loss = criterion(outputs, targets)
            epoch_loss += outputs.shape[0] * loss.item()

            acc+=torch.sum(outputs.argmax(dim=-1) == targets).item()

    acc_print = acc/len(test_dataset)
    loss_print = epoch_loss / len(test_dataset)

    print("Test Loss:", loss_print)
    print('Test Accuracy:{0:.3%}'.format(acc_print))

    f = open("./models/statistics.txt", "a")
    text_test = "Test Loss: "+ str(loss_print) + ", " + "Test Accuracy: " + str(acc_print) + "\n"
    f.write(text_test)
    f.close()

    sent_results(text=text_test)

    #del samples
    #del targets

### Sent results to an email (optional):

In [14]:
from utils import sent_results

# Test:
sent_results(text='Start')

'Email sent successfully!'

## Prepare a model

In [16]:
model = deit_base_patch16_224(pretrained=True, use_top_n_heads=12,use_patch_outputs=False).cuda()

# # Freeze backbone:
for param in model.parameters():
    param.requires_grad = False #True

# Add linear classifier on top:
model.head = torch.nn.Linear(in_features=model.head.in_features, out_features=classes_number)
model.head.apply(model._init_weights)
for param in model.head.parameters():
    param.requires_grad = True

_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])


Linear(in_features=6144, out_features=251, bias=True)

In [14]:
epochs = 30
#change criterion here
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.999))

model = model.to(device)

for epoch in range(epochs):
    train (device=device, model=model, criterion=criterion, optimizer=optimizer, 
        train_dataset=train_dataset, train_loader=train_loader, epoch=epoch)

    test (device=device, model=model, criterion=criterion, 
        test_dataset=test_dataset, test_loader=test_loader, test_only = False) #, model_path=model_path)

Training....


100%|██████████| 462/462 [42:13<00:00,  5.48s/it]


Epoch: 1 | Loss: 2.602127376442897
Train Accuracy:42.697%
Testing....


100%|██████████| 375/375 [01:32<00:00,  4.07it/s]


Test Loss: 1.8005262642318614
Test Accuracy:55.544%
Training....


...

Test Loss: 2.399460998320154
Test Accuracy:55.386%
Training....


100%|██████████| 462/462 [15:36<00:00,  2.03s/it]


Epoch: 30 | Loss: 0.29862291540921454
Train Accuracy:92.140%
Testing....


100%|██████████| 375/375 [01:23<00:00,  4.50it/s]

Test Loss: 2.429801371210075
Test Accuracy:55.278%


# Test only:
To check a specific model

In [15]:
# It is assumed that a model is defined already
model_path = './models/model_3ep_1.4loss.pt'
criterion = torch.nn.CrossEntropyLoss()

test (device=device, model=model, criterion=criterion, 
    test_dataset=test_dataset, test_loader=test_loader, test_only = True, model_path=model_path)

Testing....
Test only!


100%|██████████| 375/375 [01:26<00:00,  4.31it/s]


Test Loss: 1.7027152970530142
Test Accuracy:58.463%
